In [17]:
import burglaries_per_month_by_lsoa
import xgboost as xgb
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [18]:
data = burglaries_per_month_by_lsoa.main()

100%|██████████| 211/211 [00:01<00:00, 158.33it/s]


In [20]:
mseDct = {}
importanceDF = pd.DataFrame()

for key in tqdm(data.keys()):
    burglaries, unemployement = data[key]
    burglaries = burglaries.reset_index()
    burglaries = burglaries.rename(columns={0:"BPM"})
    burglaries = burglaries.set_index("date")
    burglaries = burglaries.join(unemployement, how="inner")
    burglaries.sort_index(inplace=True)
    burglaries["PCT_value"] = burglaries["value"].pct_change()
    burglaries["lag1_pct"] = burglaries["PCT_value"].shift(1)
    burglaries["lag2_pct"] = burglaries["PCT_value"].shift(2)
    burglaries["lag3_pct"] = burglaries["PCT_value"].shift(3)
    burglaries["lag4_pct"] = burglaries["PCT_value"].shift(4)
    burglaries["lag5_pct"] = burglaries["PCT_value"].shift(5)
    burglaries["lag6_pct"] = burglaries["PCT_value"].shift(6)
    burglaries = burglaries[burglaries.index<"2020"]
    burglaries = burglaries.fillna(0)
    burglaries.replace([np.inf, -np.inf], 0, inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(burglaries[["PCT_value", "lag1_pct","lag2_pct","lag3_pct", "lag4_pct", "lag5_pct", "lag6_pct"]].copy(), burglaries["BPM"].copy(), test_size=0.1)
    model = xgb.XGBRegressor(booster = "gblinear")
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mseDct[key] = mean_squared_error(y_test, predictions)
    importance_scores = model.coef_
    importance_dict = dict(zip(X_train.columns,importance_scores))


  0%|          | 0/211 [00:00<?, ?it/s]

In [12]:
for key in data.keys():
    a, b = data[key]
    break


TypeError: 'dict_keys' object is not an iterator

In [24]:
import matplotlib.pyplot as plt

type(importance_dict)

dict

In [ ]:
df = pd.DataFrame(columns=)
for code in importance_dict:
    for key in importance_dict.keys():
